In [2]:
import pandas as pd
import requests
import json

In [18]:
# Request input from user
message=str.lower(input('Enter message:'))

Enter message:if i can't let it go out of my mind


In [21]:
encoded_key='NGM1MzdmYzQ2OTdjNDUwOGIzMzFhY2E4NzlkNWU1ZjI6NzU4ZjA2ZjhkOWU5NGM2MGFmZGRkNGFhN2MwMjkwOGY='
def client_credentials(encoded_key):
    headers = {
    'Authorization': 'Basic '+encoded_key,
    }

    data = {
      'grant_type': 'client_credentials'
    }

    response = requests.post('https://accounts.spotify.com/api/token', headers=headers, data=data)
#     print(response.json())
#     {'access_token': 'BQCVHHb7qAYICBc8He7AJi-SfHKWHZZQWSJ0fdetTyXFISQZR-iTIt1Y2-33XYk-LqKT_0YqrJ4TCnlVZIQ', 
#      'token_type': 'Bearer', 'expires_in': 3600, 'scope': ''}
    try:
        return response.json()['access_token']
    except:
        print(response.json())
        return response.json()
    

access_token=client_credentials(encoded_key)

In [5]:
offset=0
limit=50

In [16]:
def search_exact_track(q,n,k):
    # grab all 50 searches, see if any match exactly
    # if not, re-iterate next 50 searches
    
    print('Offset: '+str(n))
    if offset/limit>7:
        print('Too many iterations to find the track')
        return None
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'Authorization': 'Bearer '+access_token,
    }

    params = (
        ('q', q),
        ('type', 'track'),
        ('market', 'US'),
        ('offset',n),
        ('limit',k)
    )
    
    response = requests.get('https://api.spotify.com/v1/search', headers=headers, params=params)
    tracks=response.json()
    if tracks['tracks']['total']==0:
        print('No matching tracks for: '+q)
        return None
    track_dict={str.lower(x['name']):x['external_urls']['spotify'] for x in list(tracks['tracks']['items'])}
    if q in track_dict:
        print('Found track: '+q)
        return(track_dict[q])
    else:
        print('Must reiterate with new offset: '+str(n+k))
        return(search_tracks(q,n+k,k))



In [17]:
search_exact_track(message, offset, limit)

Offset: 0
Must reiterate with new offset: 50
Offset: 50
Must reiterate with new offset: 100
Offset: 100
Found track: hi


'https://open.spotify.com/track/0GelEhEgigmyK5jGXbPhAg'

In [20]:
message.split(" ")

['if', 'i', "can't", 'let', 'it', 'go', 'out', 'of', 'my', 'mind']